# Random Forest - Classification Step 1 (Unbalanced dataset)

In [2]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

In [3]:
# importing dataset
ids_dataset = pd.read_csv("IDS_merged_dataset.csv")
ids_dataset.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_12656\1319772336.py:2: DtypeWarning: Columns (0,1,3,6,84) have mixed types. Specify dtype option on import or set low_memory=False.
  ids_dataset = pd.read_csv("IDS_merged_dataset.csv")


,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.5-8.254.250.126-49188-80-6,8.254.250.126,80.0,192.168.10.5,49188.0,6.0,03/07/2017 08:55:58,4.0,2.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,192.168.10.5-8.254.250.126-49188-80-6,8.254.250.126,80.0,192.168.10.5,49188.0,6.0,03/07/2017 08:55:58,1.0,2.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,192.168.10.5-8.254.250.126-49188-80-6,8.254.250.126,80.0,192.168.10.5,49188.0,6.0,03/07/2017 08:55:58,1.0,2.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,192.168.10.5-8.254.250.126-49188-80-6,8.254.250.126,80.0,192.168.10.5,49188.0,6.0,03/07/2017 08:55:58,1.0,2.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,192.168.10.14-8.253.185.121-49486-80-6,8.253.185.121,80.0,192.168.10.14,49486.0,6.0,03/07/2017 08:56:22,3.0,2.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [4]:
# Classes
ids_dataset[' Label'].unique()

array(['BENIGN', 'FTP-Patator', 'SSH-Patator', 'DoS slowloris',
       'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye', 'Heartbleed',
       'Infiltration', 'Web Attack – Brute Force', 'Web Attack – XSS',
       'Web Attack – Sql Injection', nan, 'Bot', 'DDoS', 'PortScan'],
      dtype=object)

In [5]:
# filtering dataset;- selecting only BEGNIN records
# ids_dataset = ids_dataset[ids_dataset[' Label'] == 'BENIGN']
# ids_dataset.head()

In [6]:
ids_dataset.shape

(3119345, 85)

In [7]:
# dataset information
# ids_dataset.info()

In [8]:
# Dataset columns
# ids_dataset.columns

In [9]:
# # dropping unwanted columns
# columns_to_be_dropped = [
#     'Flow ID', 
#     ' Source IP', 
#     ' Source Port',
#     ' Destination IP', 
#     ' Destination Port', 
#     ' Protocol',
#     ' Timestamp',
#     ' Label'
# ]

# pre_pca_ids_dataset = ids_dataset.drop(columns_to_be_dropped, axis=1)
# pre_pca_ids_dataset.head()

In [10]:
selected_features = [' Bwd Packet Length Std', ' Packet Length Variance',
       'Init_Win_bytes_forward', 'Bwd Packet Length Max',
       ' Average Packet Size', ' Packet Length Std',
       ' Packet Length Mean', ' Bwd Packet Length Mean', ' Label']
selected_dataset = ids_dataset[selected_features].copy()
selected_dataset.head()

,Bwd Packet Length Std,Packet Length Variance,Init_Win_bytes_forward,Bwd Packet Length Max,Average Packet Size,Packet Length Std,Packet Length Mean,Bwd Packet Length Mean,Label
0,0.0,0.0,329.0,0.0,9.0,0.0,6.0,0.0,BENIGN
1,0.0,0.0,329.0,0.0,9.0,0.0,6.0,0.0,BENIGN
2,0.0,0.0,329.0,0.0,9.0,0.0,6.0,0.0,BENIGN
3,0.0,0.0,329.0,0.0,9.0,0.0,6.0,0.0,BENIGN
4,0.0,0.0,245.0,0.0,9.0,0.0,6.0,0.0,BENIGN


In [11]:
# 5 summary of dataset
selected_dataset.describe()

,Bwd Packet Length Std,Packet Length Variance,Init_Win_bytes_forward,Bwd Packet Length Max,Average Packet Size,Packet Length Std,Packet Length Mean,Bwd Packet Length Mean
count,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06
mean,3.353257e+02,4.861548e+05,6.989837e+03,8.708495e+02,1.919837e+02,2.949756e+02,1.719444e+02,3.059493e+02
std,8.396932e+02,1.647490e+06,1.433873e+04,1.946367e+03,3.318603e+02,6.318001e+02,3.054915e+02,6.052568e+02
min,0.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,7.500000e+00,0.000000e+00,6.000000e+00,0.000000e+00
50%,0.000000e+00,6.750000e+02,2.510000e+02,7.900000e+01,7.225000e+01,2.598076e+01,5.720000e+01,7.200000e+01
75%,7.794054e+01,3.038884e+04,8.192000e+03,2.800000e+02,1.492639e+02,1.743239e+02,1.198000e+02,1.810000e+02
max,8.194660e+03,2.240000e+07,6.553500e+04,1.953000e+04,3.893333e+03,4.731522e+03,3.337143e+03,5.800500e+03


In [12]:
selected_dataset.shape

(3119345, 9)

In [13]:
# Dropping missing values
selected_dataset.dropna(inplace=True)

In [14]:
selected_dataset.shape

(2830743, 9)

In [15]:
# Converting infinity to nan
selected_dataset.replace([np.inf, -np.inf], np.nan, inplace=True)

In [16]:
# Dropping na
selected_dataset.dropna(inplace=True)

In [17]:
selected_dataset.shape

(2830743, 9)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    selected_dataset.drop(' Label', axis=1), 
    selected_dataset[' Label'],
    test_size=0.2,
    random_state=42)

In [19]:
# random forest object
rf_model = RandomForestClassifier()

In [20]:
selected_dataset.columns

Index([' Bwd Packet Length Std', ' Packet Length Variance',
       'Init_Win_bytes_forward', 'Bwd Packet Length Max',
       ' Average Packet Size', ' Packet Length Std', ' Packet Length Mean',
       ' Bwd Packet Length Mean', ' Label'],
      dtype='object')

In [21]:
# fitting model
rf_model.fit(X_train, y_train)

RandomForestClassifier()

In [22]:
# model evaluation
y_pred = rf_model.predict(X_test)

In [23]:
pd.DataFrame(confusion_matrix(y_test, y_pred))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,453623,3,5,167,703,34,0,10,0,0,307,86,0,0,2
1,227,146,0,0,0,0,0,0,0,0,0,0,0,0,0
2,36,0,25455,0,1,0,0,0,0,0,0,0,0,0,0
3,14,0,0,1973,0,1,0,0,0,0,0,0,0,0,0
4,495,0,0,37,45622,0,0,0,0,0,0,0,0,0,0
5,563,0,0,0,0,531,6,2,0,0,0,0,0,0,0
6,323,0,0,1,0,0,824,0,0,0,0,0,0,0,0
7,6,0,0,0,0,0,0,1593,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0
9,5,0,0,0,0,0,0,0,0,3,0,0,0,0,0


In [25]:
print(classification_report(y_test, y_pred))

                            precision    recall  f1-score   support

                    BENIGN       1.00      1.00      1.00    454940
                       Bot       0.98      0.39      0.56       373
                      DDoS       1.00      1.00      1.00     25492
             DoS GoldenEye       0.91      0.99      0.95      1988
                  DoS Hulk       0.98      0.99      0.99     46154
          DoS Slowhttptest       0.94      0.48      0.64      1102
             DoS slowloris       0.99      0.72      0.83      1148
               FTP-Patator       0.99      1.00      0.99      1599
                Heartbleed       1.00      1.00      1.00         3
              Infiltration       1.00      0.38      0.55         8
                  PortScan       0.99      1.00      0.99     31740
               SSH-Patator       0.93      1.00      0.96      1162
  Web Attack – Brute Force       1.00      0.09      0.17       297
Web Attack – Sql Injection       1.00      0.33